# How to run the program
To run this notebook, click kernel and run all. This should install the dependancies, start a camera feed which will save data on hand features it is given and will wait for you to press '.' on the notebook to save the data to a CSV file which will be used in the 'Building a model' notebook.

# Dependacies

Installing required libaries using pip.

In [1]:
#Un-comment if installing is required

!pip install opencv-python
!pip install mediapipe
!pip install keyboard


# Import Required Libaries
Setting up the imports for the project and populating an array with the headers for the data about to be captured and a key press boolean for recording keyboard inputs for exporting the array to a csv file.

In [2]:
#Importing the required libaries

import cv2
import mediapipe as mp
import time
import os
import sys
import keyboard
import time

csv = 'X_0, Y_0, X_1, Y_1, X_2, Y_2, X_3, Y_3, X_4, Y_4, X_5, Y_5, X_6, Y_6, X_7, Y_7, X_8, Y_8, X_9, Y_9, X_10, Y_10, X_11, Y_11, X_12, Y_12, X_13, Y_13, X_14, Y_14, X_15, Y_15, X_16, Y_16, X_17, Y_17, X_18, Y_18, X_19, Y_19, X_20, Y_20, Label\n'

keydown = False


# Hand Detector Class

Creating a hand detector class for reusability. Handles using OpenCV model to location hand landmarks and return position information for later use.

In [3]:
#Creating a hand detector class to allow this it be used for multiple projects

class handDetector():
    
    def __init__(self, mode=False, Hands =2, Complexity=1, detectionConf=0.5, trackConf=0.5):
        
        self.mode = mode
        self.Hands = Hands
        self.Complexity = Complexity
        self.detectionConf = detectionConf
        self.trackConf = trackConf
        
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.Hands, self.Complexity, self.detectionConf, self.trackConf)
        self.mpDraw = mp.solutions.drawing_utils


    def findHands(self, img, draw=True):
        
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, 
                                               self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handNo=0, draw=True):
        
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            
            for id, lm in enumerate(myHand.landmark):

                    h ,w, c = img.shape
                    cx, cy = int(lm.x*w), int(lm.y*h)
                    lmList.append([id, cx, cy])
                    if draw:
                        cv2.circle(img, (cx,cy), 15, (250, 0, 250), cv2.FILLED)
                    
        return lmList


# Logic Classification for Data Logging
Using logical classification to create a dataset for later model training, this is purely used for creating data quickly with accurate data labeling. Also used for drawing the logic's classification to the webcam image as a overlay to ensure it is working correctly when in operation. On each loop, the method checks to see if the '.' key has been pressed on the notebook to indicate that the data should be exported.

In [7]:
def finger_counter():
    
    global keydown

    wCam, hCam = 640, 480

    cap = cv2.VideoCapture(0)
    cap.set(3, wCam)
    cap.set(4, hCam)

    folderPath = "digitimg"
    myList = os.listdir(folderPath)

    print(myList)


    overlayList = []
    for imPath in myList:
        image = cv2.imread(f'{folderPath}/{imPath}')
        overlayList.append(image)

    print(len(overlayList))
    pTime = 0


    detector = handDetector(detectionConf=0.75)

    tipIds = [ 4, 8, 12, 16, 20]


    while True:

        success, img = cap.read()
        img = detector.findHands(img)
        lmList = detector.findPosition(img, draw=False)

        if len(lmList) !=0:
            fingers = []

            # thumb
            if lmList[tipIds[0]][1] > lmList[tipIds[0]-1][1]:
                fingers.append(1)
            else:
                fingers.append(0)

            # 4 fingers
            for id in range(1,5):
                if lmList[tipIds[id]][2] < lmList[tipIds[id]-2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            totalFingers = fingers.count(1)

            h, w, c = overlayList[totalFingers-1].shape
            img[0:h, 0:w] = overlayList[totalFingers-1]

            cv2.rectangle(img, (29,255),(170,425),(0,255,0), cv2.FILLED)
            cv2.putText(img, str(totalFingers),(45,375),cv2.FONT_HERSHEY_PLAIN,10,(255,0,0),25)

        cTime = time.time()
        fps = 1/(cTime - pTime)
        pTime = cTime



        if len(lmList)!=0:
            storelmtomemory(lmList, totalFingers)
        
        try:  # used try so that if user pressed other than the given key error will not be shown
            if keyboard.is_pressed('.'):  # if key '.' is pressed 
                
                if keydown == False:
                    keydown = True
                    writememorytocsv()
                    time.sleep(0.5)
                    keydown = False
        except:
            x = 1


        cv2.putText(img, f'FPS: {int(fps)}', (400, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)

        cv2.imshow("Image", img)
        cv2.waitKey(1)


# Storing Data
storelmtomemory - Used to pass each data line to a buffer array to hold all the data captured in the session

writememorytocsv - Takes the data held by 'storelmtomemory' and writes this data to a CSV file to be used in the 'Building a model' Notebook.

In [5]:
def storelmtomemory(lmList, totalFingers):
    global csv
    for lm in lmList:
        csv += f'{lm[1]},{lm[2]},'
        
    csv += f'{totalFingers}\n'
    
def  writememorytocsv():
    dt = time.strftime("%Y%m%d-%H%M%S")
    f = open(f'handdata_{dt}.csv', "a")
    f.write(csv)
    f.close()
    print('Wrote to file.')
    

# Main Method
Finger_counter creates a while loop camera feed that only adds to the buffer array when a hand is detected. Data will continue to be recorded into this buffer until the notepad detects the '.' key pressed, which will then write the buffer to a csv file.

In [ ]:

try:
    finger_counter()
except KeyboardInterrupt:
    print('Interrupted')
try:
    sys.exit(0)
except SystemExit:
    os._exit(0)
    







['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg']
6
Wrote to file.
